# Named Entity Recognition for the decicontas.br dataset

This script implements Named Entity Recognition (NER) extraction applied to the **decicontas.br** dataset, which consists of decisions from the Rio Grande do Norte State Court of Accounts (TCE/RN) involving fines, obligations, reimbursements, and recommendations. The solution uses large language models (LLMs) deployed through Azure OpenAI, with integration via `langchain` and `pydantic` for generating structured outputs.

The goal is to evaluate the ability of LLMs to transform unstructured legal text into standardized data with consistent labels (e.g., MULTA, OBRIGACAO, RECOMENDACAO, RESSARCIMENTO), supporting downstream analysis and monitoring of audit decisions. The project is inspired by LexCare.BR (focused on health judicialization) and applies *function calling* and *few-shot prompting* to decisions from the TCE/RN.


In [1]:
import pprint
import time

import pandas as pd

from tqdm import tqdm
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv

from tools.dataset import get_decicontas_df
from tools.prompt import generate_few_shot_ner_prompts
from tools.schema import (
    NERDecisao
)

load_dotenv()

True

# Loading and Setup

The dataset is loaded using the `get_decicontas_df()` function, which wraps the loading of a CSV file annotated via Label Studio. This dataset contains excerpts from TCE/RN decisions, each annotated with named entities relevant to audit enforcement.

Six Azure OpenAI models are instantiated for evaluation:

- `gpt-35`
- `gpt-4-turbo`
- `gpt-4o`
- `gpt-41`
- `gpt-41-mini`
- `gpt-41-nano`

Each model is configured with `with_structured_output` to ensure outputs conform to the predefined `NERDecisao` JSON schema. This enforces consistent formatting, allows automatic validation, and simplifies integration with downstream evaluation tools that rely on structured span-based comparisons.


In [ ]:
df_decicontas = get_decicontas_df()

In [21]:
gpt41_nano = AzureChatOpenAI(
    deployment_name="gpt-4-1-nano",  
    model_name="gpt-4-1-nano"
)

gpt41_mini = AzureChatOpenAI(
    deployment_name="gpt-4-1-mini",  
    model_name="gpt-4-1-mini"
)

gpt41= AzureChatOpenAI(
    deployment_name="gpt-4-1",  
    model_name="gpt-4-1"
)

# Inicializadores com extração estruturada
extractor_gpt41_nano = gpt41_nano.with_structured_output(NERDecisao, include_raw=False, method="function_calling")
extractor_gpt41_mini = gpt41_mini.with_structured_output(NERDecisao, include_raw=False, method="function_calling")
extractor_gpt41 = gpt41.with_structured_output(NERDecisao, include_raw=False, method="function_calling")


gpt4o = AzureChatOpenAI(
    deployment_name="gpt-4o",  
    model_name="gpt-4o",       
)

gpt35 = AzureChatOpenAI(
    deployment_name="gpt-35",  
    model_name="gpt-35-turbo",
)

gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

extractor_gpt4o = gpt4o.with_structured_output(NERDecisao, include_raw=False, method="function_calling")
extractor_gpt35 = gpt35.with_structured_output(NERDecisao, include_raw=False, method="function_calling")
extractor_gpt4_turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="function_calling")

In [26]:
df_decicontas["checked"] = False

# Exploratory Data Analysis (EDA)

Before running the NER pipeline, an exploratory data analysis was carried out to inspect the distribution of the labels previously annotated in the dataset. This investigation serves to:

- validate consistency of the manual annotations
- identify classes with low representation
- guide adjustments for class balance during evaluation

The frequency of each label was extracted by scanning the Label Studio annotation results and summarized in a DataFrame.


In [3]:
labels_seen = []
for i,r in df_decicontas.iterrows():
    for a in r['annotations']:
        if 'result' in a.keys():
            for v in a['result']:
                labels_seen.append(v['value']['labels'][0])    
pd.Series(labels_seen).value_counts().sort_index().to_frame('count').reset_index().rename(columns={'index': 'label'}).sort_values(by='count', ascending=False)

label  count
0          MULTA    204
1      OBRIGACAO    120
3  RESSARCIMENTO     63
2   RECOMENDACAO     58

# Example use

To validate the pipeline, a test query was built with real excerpts from TCE/RN decisions, simulating fines, deadlines, and enforcement clauses. This sample text was passed to the few-shot prompt generation function to check the schema, ensure consistent attribute definitions, and manually verify correctness before applying the pipeline to the full dataset.

The generated prompt was displayed with `pprint` for manual inspection and debugging.


In [4]:
EXAMPLE_TEXT = '''
DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, em consonância com a informação do Corpo Técnico e com o parecer do Ministério Público que atua junto a esta Corte de Contas, acolhendo integralmente o voto do Conselheiro Relator, julgar: a) pela DENEGAÇÃO DE REGISTRO ao ato concessivo da aposentadoria e à despesa dele decorrente; b) pela determinação ao IPERN, à vista da Lei Complementar Estadual nº 547/2015, para que, no prazo de 60 (sessenta) dias, após o trânsito em julgado desta decisão, adote as correções necessárias para regularização do ato concessório, do cálculo dos proventos e de sua respectiva implantação; c) no caso de descumprimento da presente decisão, a responsabilização do titular da pasta responsável por seu atendimento, sem prejuízo da multa cominatória desde já fixada no valor de R$ 50,00 (cinquenta reais) por dia que superar o interregno fixado no item `b`, com base no art. 110 da Lei Complementar Estadual nº 464/2012, valor este passível de revisão e limitado ao teto previsto no art. 323, inciso II, alínea `f`, do Regimento Interno, a ser apurado por ocasião de eventual subsistência de mora.
'''
prompt_with_few_shot = generate_few_shot_ner_prompts(EXAMPLE_TEXT)
pprint.pprint(prompt_with_few_shot)


ChatPromptValue(messages=[SystemMessage(content='Você é um especialista em extração de entidades nomeadas com precisão excepcional. Sua tarefa é identificar e extrair informações específicas do texto fornecido, seguindo estas diretrizes:\n\n1. Extraia as informações exatamente como aparecem no texto, sem interpretações ou alterações.\n2. Se uma informação solicitada não estiver presente ou for ambígua, retorne null para esse campo.\n3. Mantenha-se estritamente dentro do escopo das entidades e atributos definidos no esquema fornecido.\n4. Preste atenção especial para manter a mesma ortografia, pontuação e formatação das informações extraídas.\n5. Não infira ou adicione informações que não estejam explicitamente presentes no texto.\n6. Se houver múltiplas menções da mesma entidade, extraia todas as ocorrências relevantes.\n7. Ignore informações irrelevantes ou fora do contexto das entidades solicitadas.\n\n**Orientação adicional para OBRIGACAO**: considere apenas o dispositivo da decisão

# Evaluating the models

In this stage, the script loops over the entire dataset to apply batch NER inference. Each model (gpt-4o, gpt-4-turbo, and gpt-35-turbo) receives the few-shot prompts and returns structured predictions. These results are stored along with the source text and reference (“golden”) annotations to enable metric evaluation later.

The intermediate results are saved in JSON so they can be reused without requiring the models to be rerun, which saves time and reduces costs.


In [140]:
MODELS = [
    ('gpt-41', extractor_gpt41),
    ('gpt-41-nano', extractor_gpt41_nano),
    ('gpt-41-mini', extractor_gpt41_mini),
    ('gpt-35', extractor_gpt35),
    ('gpt-4o', extractor_gpt4o),
    ('gpt-4-turbo', extractor_gpt4_turbo)
]
models_names = [x[0] for x in MODELS]


In [141]:
models_names

['gpt-41', 'gpt-41-nano', 'gpt-41-mini', 'gpt-35', 'gpt-4o', 'gpt-4-turbo']

In [58]:
model_index = []
for m in models_names:
    model_index.extend(list(zip([m] * len(df_decicontas.index), df_decicontas.index)))
df_checked = pd.DataFrame(model_index, columns=['model', 'index'])
df_checked["checked"] = False

In [63]:
len(df_decicontas.index), len(df_checked.index), len(models_names)

(1425, 4275, 3)

In [50]:
models_results = []

df_results = pd.DataFrame(columns=['index', 'text', 'pred', 'golden', 'model'])

In [91]:
len(df_checked[df_checked['checked']])

138

In [90]:
df_results

index                                               text  \
0       0  DECIDEM os Conselheiros do Tribunal de Contas ...   
1       1  DECIDEM os Conselheiros do Tribunal de Contas ...   
2       2  DECIDEM os Conselheiros do Tribunal de Contas ...   
3       3  DECIDEM os Conselheiros do Tribunal de Contas ...   
4       4  DECIDEM os Conselheiros do Tribunal de Contas ...   
..    ...                                                ...   
133   133  DECIDEM os Conselheiros do Tribunal de Contas ...   
134   134  DECIDEM os Conselheiros do Tribunal de Contas ...   
135   135  DECIDEM os Conselheiros do Tribunal de Contas ...   
136   136  DECIDEM os Conselheiros do Tribunal de Contas ...   
137   137  DECIDEM os Conselheiros do Tribunal de Contas ...   

                                                  pred golden   model  
0    {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
1    {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
2    {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
3    {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
4    {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
..                                                 ...    ...     ...  
133  {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
134  {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
135  {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
136  {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  
137  {'multas': [], 'ressarcimentos': [], 'obrigaco...     []  gpt-41  

[138 rows x 5 columns]

In [ ]:
for model_name, model_extractor in MODELS:
    print(f"Extracting {model_name} results")

    df_results = pd.DataFrame(columns=['index', 'text', 'pred', 'golden', 'model'])

    rows_to_process = [
        (index, row) for index, row in df_decicontas.iterrows()
        if not df_checked.loc[(df_checked['model'] == model_name) & (df_checked['index'] == index), 'checked'].values[0]
    ]

    for index, row in tqdm(rows_to_process, desc=f"{model_name}", unit="instance"):
        prompt_with_few_shot = generate_few_shot_ner_prompts(row['data']['text'])

        success = False
        while not success:
            try:
                result = model_extractor.invoke(prompt_with_few_shot)
                success = True
            except Exception as e:
                print(f"Error at index {index}: {e}")
                time.sleep(3)

        extracted_result = pd.DataFrame([{
            'index': index,
            'text': row['data']['text'],
            'pred': result.model_dump(),
            'golden': [r['value'] for r in row['annotations'][0]['result']],
            'model': model_name
        }])
        df_results = pd.concat([df_results, extracted_result], ignore_index=True)
        
        df_checked.loc[(df_checked['model'] == model_name) & (df_checked['index'] == index), 'checked'] = True
    
    df_results.to_json(f"dataset/labeled_data/models_results_decicontas_{model_name}.json", orient="records", force_ascii=False, indent=2)

Extracting gpt-41 results


gpt-41: 0instance [00:00, ?instance/s]


Extracting gpt-41-nano results


gpt-41-nano:   1%|▏         | 8/571 [00:18<13:32,  1.44s/instance]  

Error at index 862: 1 validation error for NERDecisao
recomendacoes.0
  Input should be a valid dictionary or instance of NERRecomendacao [type=model_type, input_value='Destaca a desnecessidade... à Justiça Eleitoral.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
Error at index 862: 1 validation error for NERDecisao
recomendacoes.0
  Input should be a valid dictionary or instance of NERRecomendacao [type=model_type, input_value='destaca a desnecessidade... à Justiça Eleitoral.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
Error at index 862: 1 validation error for NERDecisao
recomendacoes.0
  Input should be a valid dictionary or instance of NERRecomendacao [type=model_type, input_value='Destaca a desnecessidade... à Justiça Eleitoral.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
Error at index 862: 1 validation error for NERDecisa

...

gpt-41-nano:  85%|████████▍ | 483/571 [09:55<01:31,  1.04s/instance]

Error at index 1337: 1 validation error for NERDecisao
multas.1
  Input should be a valid dictionary or instance of NERMulta [type=model_type, input_value='{', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type


gpt-41-nano: 100%|██████████| 571/571 [13:34<00:00,  1.43s/instance]


Extracting gpt-41-mini results


gpt-41-mini: 100%|██████████| 1425/1425 [38:31<00:00,  1.62s/instance] 


# Assessing the metrics

To measure the models’ performance, the script uses two main evaluation strategies:

- **Token-level (seqeval)**: based on token segmentation, measuring precision, recall, and F1 using the IOB scheme.
- **Span-level IoU (Intersection over Union)**: computing the overlap between predicted spans and gold-standard spans, taking entity type into account. A threshold of IoU ≥ 0.5 is considered a valid match.

Additionally, the script reports aggregate metrics and also details them by label, showing the number of matching spans between predictions and ground truth. This allows a clearer view of each model’s behavior per entity type (MULTA, OBRIGACAO, RECOMENDACAO, RESSARCIMENTO).


In [134]:
df_models = pd.concat([pd.read_json(f"dataset/labeled_data/models_results_decicontas_{model_name}.json") for model_name in models_names], ignore_index=True)

In [135]:
len(df_models), df_models['model'].unique()

(8550,
 array(['gpt-41', 'gpt-41-nano', 'gpt-41-mini', 'gpt-35', 'gpt-4o',
        'gpt-4-turbo'], dtype=object))

In [136]:
from rapidfuzz import fuzz

DICT_LABELS = {
    "obrigacoes": "OBRIGACAO",
    "recomendacoes": "RECOMENDACAO",
    "ressarcimentos": "RESSARCIMENTO",
    "multas": "MULTA",
}

def convert_pred_to_golden_format(row, window_size=500, step_size=100, min_score=80):
    pred_spans = []
    text = row['text']
    pred = row['pred']
    
    for label_type, spans in pred.items():
        for span in spans:
            if not isinstance(span, dict):
                continue
            span_text = (
                span.get("descricao_multa")
                or span.get("descricao_obrigacao")
                or span.get("descricao_ressarcimento")
                or span.get("descricao_recomendacao")
            )
            if not span_text:
                continue
            
            best_score = 0
            best_pos = -1
            best_substring = ""
            
            # sliding window search
            for start in range(0, len(text), step_size):
                window = text[start:start+window_size]
                score = fuzz.partial_ratio(span_text, window)
                if score > best_score and score >= min_score:
                    best_score = score
                    best_pos = start + window.find(span_text.split()[0]) if span_text.split() else start
                    best_substring = span_text
            
            if best_score >= min_score and best_pos >= 0:
                pred_spans.append({
                    "start": best_pos,
                    "end": best_pos + len(best_substring),
                    "text": best_substring,
                    "labels": [DICT_LABELS[label_type]]
                })
                
    return pred_spans


In [137]:
df_models['pred_as_golden'] = df_models.apply(
    lambda row: convert_pred_to_golden_format(row, window_size=500, step_size=100, min_score=80),
    axis=1
)

In [138]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from collections import defaultdict

def compute_iou_score(span_a, span_b, label_a, label_b, threshold=0.5):
    """
    Computes the IoU agreement score between two spans with labels,
    as described in the definition you gave.
    """
    s_a, e_a = span_a
    s_b, e_b = span_b

    # no overlap at all
    if e_a < s_b or e_b < s_a:
        return 0.0
    
    intersection = max(0, min(e_a, e_b) - max(s_a, s_b))
    union = max(e_a, e_b) - min(s_a, s_b)
    
    iou = intersection / union if union > 0 else 0.0

    if iou >= threshold:
        delta = 1 if label_a == label_b else 0
        return iou * delta
    else:
        return 0.0

def calculate_metrics(df, iou_threshold=0.5):
    from collections import defaultdict
    from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

    y_true = []
    y_pred = []
    label_metrics = defaultdict(lambda: {"total_gold": 0, "total_pred": 0, "matched": 0})

    for _, row in df.iterrows():
        text = row['text']
        true_labels = ['O'] * len(text)
        pred_labels = ['O'] * len(text)

        # gold
        for ann in row['golden']:
            start, end, label = ann['start'], ann['end'], ann['labels'][0]
            if start < len(true_labels):
                true_labels[start] = f"B-{label}"
                for i in range(start+1, min(end, len(true_labels))):
                    true_labels[i] = f"I-{label}"

        # pred
        for ann in row['pred_as_golden']:
            start, end, label = ann['start'], ann['end'], ann['labels'][0]
            if start < len(pred_labels):
                pred_labels[start] = f"B-{label}"
                for i in range(start+1, min(end, len(pred_labels))):
                    pred_labels[i] = f"I-{label}"

        y_true.append(true_labels)
        y_pred.append(pred_labels)

        gold_spans = [(ann['start'], ann['end'], ann['labels'][0]) for ann in row['golden']]
        pred_spans = [(ann['start'], ann['end'], ann['labels'][0]) for ann in row['pred_as_golden']]

        for g in gold_spans:
            label_metrics[g[2]]["total_gold"] += 1
        for p in pred_spans:
            label_metrics[p[2]]["total_pred"] += 1

        for p in pred_spans:
            for g in gold_spans:
                score = compute_iou_score(
                    (p[0], p[1]), (g[0], g[1]),
                    p[2], g[2],
                    threshold=iou_threshold
                )
                if score > 0:
                    label_metrics[p[2]]["matched"] += 1

    # seqeval
    token_prec = precision_score(y_true,y_pred)
    token_rec  = recall_score(y_true,y_pred)
    token_f1   = f1_score(y_true,y_pred)

    print("====== SEQEVAL TOKEN-LEVEL ======")
    print(f"Precision: {token_prec:.4f}")
    print(f"Recall:    {token_rec:.4f}")
    print(f"F1:        {token_f1:.4f}")
    print(classification_report(y_true,y_pred))

    total_gold = sum(v["total_gold"] for v in label_metrics.values())
    total_pred = sum(v["total_pred"] for v in label_metrics.values())
    total_matched = sum(v["matched"] for v in label_metrics.values())

    iou_prec = total_matched / total_pred if total_pred > 0 else 0
    iou_rec  = total_matched / total_gold if total_gold > 0 else 0
    iou_f1   = 2*iou_prec*iou_rec/(iou_prec+iou_rec) if (iou_prec+iou_rec) > 0 else 0

    print("====== SPAN-LEVEL IOU>=0.5 AGGREGATED ======")
    print(f"Precision: {iou_prec:.4f}")
    print(f"Recall:    {iou_rec:.4f}")
    print(f"F1:        {iou_f1:.4f}")

    print("====== SPAN-LEVEL IOU PER LABEL ======")
    for label, m in label_metrics.items():
        prec = m["matched"] / m["total_pred"] if m["total_pred"] > 0 else 0
        rec  = m["matched"] / m["total_gold"] if m["total_gold"] > 0 else 0
        f1   = 2*prec*rec/(prec+rec) if (prec+rec)>0 else 0
        print(f"{label}: P={prec:.4f} R={rec:.4f} F1={f1:.4f} ({m['matched']} matched)")

    return {
        "seqeval": {
            "precision": token_prec,
            "recall": token_rec,
            "f1": token_f1
        },
        "iou_agg": {
            "precision": iou_prec,
            "recall": iou_rec,
            "f1": iou_f1
        },
        "iou_per_label": dict(label_metrics)
    }


In [139]:
metrics = dict()
for m in models_names:
    print(f"**********\nCalculating metrics for model: {m}\n")
    df_model = df_models[df_models['model'] == m]
    metrics[m] = calculate_metrics(df_model, iou_threshold=0.5)
    print(f"**********")

**********
Calculating metrics for model: gpt-41

====== SEQEVAL TOKEN-LEVEL ======
Precision: 0.1209
Recall:    0.1573
F1:        0.1367
               precision    recall  f1-score   support

        MULTA       0.11      0.11      0.11       204
    OBRIGACAO       0.04      0.07      0.06       120
 RECOMENDACAO       0.32      0.55      0.41        58
RESSARCIMENTO       0.09      0.10      0.09        63

    micro avg       0.12      0.16      0.14       445
    macro avg       0.14      0.21      0.17       445
 weighted avg       0.12      0.16      0.13       445

====== SPAN-LEVEL IOU>=0.5 AGGREGATED ======
Precision: 0.6221
Recall:    0.8472
F1:        0.7174
====== SPAN-LEVEL IOU PER LABEL ======
OBRIGACAO: P=0.4706 R=0.8000 F1=0.5926 (96 matched)
MULTA: P=0.7792 R=0.8824 F1=0.8276 (180 matched)
RECOMENDACAO: P=0.5050 R=0.8793 F1=0.6415 (51 matched)
RESSARCIMENTO: P=0.7143 R=0.7937 F1=0.7519 (50 matched)
**********
**********
Calculating metrics for model: gpt-41-nano

===

# Final Metrics

The experiment evaluated six GPT-based LLMs on the `decicontas.br` dataset using both token-level (seqeval) and span-level (IoU ≥ 0.5) metrics. The table below summarizes the main results:

| Model          | Token-level F1 | Span-level F1 |
|----------------|----------------|---------------|
| gpt-4-turbo    | 0.1116         | **0.7524**    |
| gpt-4o         | 0.0978         | 0.7381        |
| gpt-41-mini    | 0.0979         | 0.7399        |
| gpt-41         | **0.1367**     | 0.7174        |
| gpt-35         | 0.1146         | 0.7171        |
| gpt-41-nano    | 0.0614         | 0.4365        |

Despite modest F1 scores at the token level—due to the strictness of the IOB tagging scheme—all models except `gpt-41-nano` achieved strong performance at the span level, with F1 scores above 0.71. Notably, `gpt-4-turbo` yielded the best overall span-level F1 (75.24%), followed closely by `gpt-41-mini` (73.99%) and `gpt-4o` (73.81%).

For specific entity types, `gpt-4-turbo` showed the highest F1 for `MULTA` (86.7%) and `RESSARCIMENTO` (74.8%), while `gpt-41-mini` achieved the best performance for `OBRIGACAO` (64.06%). The `gpt-41-nano` model underperformed across the board, especially in token-level accuracy and obligation extraction.

These findings confirm the feasibility of leveraging LLMs to pre-label legal audit decisions with high span-level accuracy, similarly to what has been done in healthcare NER tasks (e.g., LexCare.br). This reduces the need for fully manual annotation and supports scalable legal text processing workflows in Portuguese.
